# Basic `MLOps` - Deploying ML models as APIs

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).

Deploying ML models as APIs is an essential component of `MLOps` (Machine Learning Operations). `MLOps` is the practice of managing and automating the entire lifecycle of a machine learning model, including development, deployment, monitoring, and maintenance. 

Deploying machine learning models as APIs enables data scientists and engineers to share their work with other members of the team or the outside world, allowing them to easily use the model's capabilities in their own applications.

By creating an API for the machine learning model, the model can be accessed remotely and integrated into various applications with ease. This helps to create a more efficient and streamlined process for using machine learning models in production, ensuring the model is up-to-date and available for use when required.

In this notebook, we will be making requests to an API that can be launched from the `xgboost_api.py` file. In it. We created an API using `FastAPI` and `Uvicorn`.

FastAPI is a modern, fast (high-performance) web framework for building APIs with Python. It is built on top of Starlette for the web parts and Pydantic for the data parts. FastAPI allows you to quickly build APIs using asynchronous operations, providing automatic validation of request parameters, data serialization, among other features.

`Uvicorn` is a lightning-fast `ASGI` (_Asynchronous Server Gateway Interface_) server that allows for asynchronous processing in web applications built with frameworks like `FastAPI`. It is built on top of the `asyncio` framework and provides an easy-to-use interface for running ASGI applications in production.

<img src="https://camo.githubusercontent.com/86d9ca3437f5034da052cf0fd398299292aab0e4479b58c20f2fc37dd8ccbe05/68747470733a2f2f666173746170692e7469616e676f6c6f2e636f6d2f696d672f6c6f676f2d6d617267696e2f6c6f676f2d7465616c2e706e67" alt="drawing" width="400"/>

We are using the same `xgboost` model we explored in [this notebook](https://github.com/Nkluge-correa/teeny-tiny_castle/blob/fa17764aa8800c388d0d298b750c686757e0861e/ML%20Intro%20Course/14_time_series_forecasting.ipynb). The [`xgboost_api.py`](https://github.com/Nkluge-correa/teeny-tiny_castle/blob/fa17764aa8800c388d0d298b750c686757e0861e/ML%20Intro%20Course/20_xgboost_api.py) file is basically an implementation of that notebook as an API that takes a series of dates and sales, trains an `XGBoost` model, and returns the predictions for the future by a fixed amount (plus some statistical information about your data distribution).

Bellow, we are just making an HTTP request to this API, which is running in parallel whit this notebook.


In [2]:
import pprint
import requests
import pandas as pd

# The data we are going to use
df = pd.read_csv('data/time_series_data.csv')

# The API endpoint URL
url = "http://127.0.0.1:8000/predict"

# Define the input data
data = {
    "product": df.product_id[0], # name of the product
    "dates": list(df.dates), # the list of dates
    "sales": list(df.sales), # the list of sales
    "ahead": 15 # how many days ahead we want to look
}

# Send a POST request with the input data
response = requests.post(url, json=data)

pprint.pprint(response.json())

{'chocolate': {'dates': ['2023-01-03',
                         '2023-01-04',
                         '2023-01-05',
                         '2023-01-06',
                         '2023-01-07',
                         '2023-01-08',
                         '2023-01-09',
                         '2023-01-10',
                         '2023-01-11',
                         '2023-01-12',
                         '2023-01-13',
                         '2023-01-14',
                         '2023-01-15',
                         '2023-01-16',
                         '2023-01-17'],
               'sales': [123.2838363647461,
                         100.26144409179688,
                         109.02445220947266,
                         120.41613006591797,
                         125.43670654296875,
                         124.01226806640625,
                         105.70793151855469,
                         129.5361785888672,
                         105.73247528076172,
           

And it is done. We have our predictions for the future. In theory, we could send any sales history to this API, and it would generate the forecast for us. 🙃

Let us see our results.

In [3]:
from IPython.display import Markdown

results_df = pd.DataFrame({
    'dates': response.json()[next(iter(response.json()))]['dates'],
    'sales': response.json()[next(iter(response.json()))]['sales']})

display(Markdown(f"""# `{next(iter(response.json()))}` Report\n{"-"*50}

{results_df.set_index('dates').to_markdown()}

**Total Sales for the next 15 days: {results_df.sales.sum():.2f} Kg.**

## Statistics Report

- **`Mean`:** {response.json()[next(iter(response.json()))]['statistics']['mean']:.2f} Kg.
- **`Minimum`:** {response.json()[next(iter(response.json()))]['statistics']['minimum']:.2f} Kg.
- **`Maximum`:** {response.json()[next(iter(response.json()))]['statistics']['maximum']:.2f} Kg.
- **`Variance`:** {response.json()[next(iter(response.json()))]['statistics']['variance']:.2f}.
- **`Standard Deviation`:** {response.json()[next(iter(response.json()))]['statistics']['std']:.2f}.

"""))


# `chocolate` Report
--------------------------------------------------

| dates      |   sales |
|:-----------|--------:|
| 2023-01-03 | 123.284 |
| 2023-01-04 | 100.261 |
| 2023-01-05 | 109.024 |
| 2023-01-06 | 120.416 |
| 2023-01-07 | 125.437 |
| 2023-01-08 | 124.012 |
| 2023-01-09 | 105.708 |
| 2023-01-10 | 129.536 |
| 2023-01-11 | 105.732 |
| 2023-01-12 | 112.911 |
| 2023-01-13 | 110.627 |
| 2023-01-14 | 123.359 |
| 2023-01-15 | 129.917 |
| 2023-01-16 | 127.937 |
| 2023-01-17 | 123.139 |

**Total Sales for the next 15 days: 1771.30 Kg.**

## Statistics Report

- **`Mean`:** 103.35 Kg.
- **`Minimum`:** 0.00 Kg.
- **`Maximum`:** 466.00 Kg.
- **`Variance`:** 2976.16.
- **`Standard Deviation`:** 54.55.



Now, you could deploy this model as a service, using, for example, a Platform as a Service (`PaaS`) to host your `API` (like [`Render`](https://render.com/) or [`Heroku`](https://www.heroku.com)). You could also deploy an already trained ML models with a more complicated architectures or services. The general blueprint will be basically the same. However, keeping your model and `API` secure and up-to-date is one of the constant works he have in `MLOps`.

Currently, this API is hosted on `Render`(`Render` has free plans), and you can make calls to it by using the [https://teeny-tiny-api.onrender.com/predict](https://teeny-tiny-api.onrender.com). However, this is a really simple model, and should not be used for real forecasting applications.

---

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).